# Cleaning Electricity Data

## Wind Electricity Data

https://www.smartgriddashboard.com/#all

On the [Eirgrid real-time system information](https://www.eirgrid.ie/grid/real-time-system-information) website it is only possible to view information for one day at a time and download data up to one month of data. Despite extensive searching I couldn't find an official source of Eirgrid historical data. I found a [GitHub repository by Daniel Parke](https://github.com/Daniel-Parke/EirGrid_Data_Download/tree/main), who has written a very helpful python file to download all the historical data. His GitHub repository contains raw csv files for the actual amount of electricity generated, actual demand, actual amount of electricity produced by wind for every year from 2014 for all Ireland, Northern Ireland and Republic of Ireland. I will need to run his program to get the most up to date data for 2024.

As my weather data is only for the Republic of Ireland, I am only interested in the csv files for the actual amount of electricity produced by wind for the Republic of Ireland. Each csv file containing one years worth of information was downloaded from the GitHub repository. After reading the data into pandas the next task will be to merge the files vertically using pd.concat(). Before I started on the onerous task of loading and concatenating each file separately, I came across a blog post on how to [read multiple csv file into pandas](https://saturncloud.io/blog/how-to-read-multiple-csv-files-into-python-pandas-dataframe). 

The solution to reading multiple files into pandas uses the glob module. Glob is a built-in module used to retrieve files/pathnames matching a specified pattern. It uses * wild cards to make path retrieval more simple and convenient. https://www.geeksforgeeks.org/how-to-use-glob-function-to-find-files-recursively-in-python/. [Real python](https://realpython.com/get-all-files-in-directory-python/#conditional-listing-using-glob) states that glob.glob() returns a list of filenames that match a pattern, which in this case are csv files. 

```python
# Search for all csv files in the current working directory
import glob
glob.glob('*.csv')
```

In [1]:
# Import modules
import pandas as pd
import glob

In [2]:
# Find all csv files in the data/electricity directory
csv_files = glob.glob('data/electricity/*.csv')

In [3]:
# Create an empty dataframe to store the combined data
electricity_df = pd.DataFrame()

# Loop through each csv file and append contents to electricity_df
for csv_file in csv_files:
    df = pd.read_csv(csv_file, 
                     header = None, 
                     names = ['date', 'wind_actual', 'location', 'wind_value'], 
                     index_col= 'date',
                     parse_dates= ['date'],
                     usecols= ['date', 'wind_value'])
    # Concatenate the df to electricity_df
    electricity_df = pd.concat([electricity_df, df])

    # Sort the df by index
    electricity_df.sort_index(inplace= True)

electricity_df.head()

,wind_value
date,
2014-01-01 00:00:00,1020.0
2014-01-01 00:15:00,995.0
2014-01-01 00:30:00,933.0
2014-01-01 00:45:00,959.0
2014-01-01 01:00:00,921.0


In [13]:
electricity_df.shape

(350680, 1)

In [4]:
electricity_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 361152 entries, 2014-01-01 00:00:00 to 2024-01-01 21:45:00
Data columns (total 1 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   wind_value  361004 non-null  float64
dtypes: float64(1)
memory usage: 5.5 MB


In [5]:
electricity_df.isna().sum()

wind_value    148
dtype: int64

The data set has over 350,000 rows. The amount of electricity produced by wind energy is recorded every 15 min. There are surprising few rows with missing data, there are only 148 rows with missing information. 

https://saturncloud.io/blog/how-to-find-all-rows-with-nan-values-in-python-pandas/

https://stackoverflow.com/questions/43424199/display-rows-with-one-or-more-nan-values-in-pandas-dataframe

In [6]:
# View the rows with missing data
nan_rows = electricity_df[electricity_df.isna().any(axis= 1)]
nan_rows

,wind_value
date,
2014-03-30 01:00:00,NaN
2014-03-30 01:15:00,NaN
2014-03-30 01:30:00,NaN
2014-03-30 01:45:00,NaN
2015-03-29 01:00:00,NaN
...,...
2023-03-26 01:00:00,NaN
2023-03-26 01:15:00,NaN
2023-03-26 01:30:00,NaN


The missing data will be filled using the interpolate() function. [Geeksforgeeks.org](https://www.geeksforgeeks.org/interpolation-in-python/) describes interpolation is a method for generating points between known values. 

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ffill.html

Why interpolate and not ffill?

In [7]:
# Interpolation the missing rows
electricity_df.interpolate(method= 'linear', inplace= True)

In [8]:
f'There are {electricity_df.index.duplicated().sum()} duplicated rows.'

'There are 10472 duplicated rows.'

The 10472 duplicated rows are all arising from the csv files. All the downloaded csv files have data for the 1st Jan for the following year e.g. the ROI_windactual_14_Eirgrid.csv contains the data for 2014 and the 1st Jan 2015. 

https://stackoverflow.com/questions/13035764/remove-pandas-rows-with-duplicate-indices

In [9]:
# Remove any duplicated rows
electricity_df = electricity_df[~electricity_df.index.duplicated(keep= 'first')]
electricity_df.head()

,wind_value
date,
2014-01-01 00:00:00,1020.0
2014-01-01 00:15:00,995.0
2014-01-01 00:30:00,933.0
2014-01-01 00:45:00,959.0
2014-01-01 01:00:00,921.0


In [14]:
electricity_df.shape

(350680, 1)

In [10]:
# Write to csv file
electricity_df.to_csv('data/electricity/clean_data/electricity_data.csv')

The electricity data is recorded every 15 min. However, the weather data is recorded hourly therefore the electricity data will be resampled to hourly.

In [15]:
hourly_electricity_df = electricity_df.resample('h').mean()
hourly_electricity_df.head()

,wind_value
date,
2014-01-01 00:00:00,976.75
2014-01-01 01:00:00,914.25
2014-01-01 02:00:00,938.50
2014-01-01 03:00:00,911.25
2014-01-01 04:00:00,915.00


In [16]:
hourly_electricity_df.to_csv('data/electricity/clean_data/hourly_electricity.csv')